<a href="https://colab.research.google.com/github/RohanRathaur1/AAI2025/blob/main/Coding_Exercise_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Customer Support Prompt Chain (Colab-ready)
# @markdown - Runs in **mock mode** by default (no API key needed).
# @markdown - Set `USE_MOCK = False` and provide `OPENAI_API_KEY` to use a real model.

from __future__ import annotations
import os, json, re
from dataclasses import dataclass

# ===================== CONFIG =====================
USE_MOCK = True  # <- set to False to call a real LLM (OpenAI example below)
OPENAI_MODEL = "gpt-4o-mini"
SYSTEM = "You are a helpful, concise customer support assistant for a company called 'NimbusNet'."

# ===================== LLM HELPER =====================
def call_llm(messages, model=OPENAI_MODEL, temperature=0.2) -> str:
    """
    Replace with your provider if desired. In mock mode, returns canned outputs.
    """
    if USE_MOCK:
        return _mock_llm(messages)

    # ---- Real OpenAI call (optional) ----
    # pip install openai
    try:
        from openai import OpenAI
    except Exception as e:
        raise RuntimeError("Install `openai` first: pip install openai") from e

    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("Please set OPENAI_API_KEY environment variable.")

    client = OpenAI(api_key=api_key)
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return resp.choices[0].message.content

# ===================== MOCK (NO API NEEDED) =====================
def _mock_llm(messages) -> str:
    """
    Minimal mock so this runs in Colab without internet or keys.
    Looks at the last user prompt to return plausible outputs.
    """
    last = [m for m in messages if m["role"] == "user"][-1]["content"]

    if "Classify the user's problem" in last:
        return json.dumps({
            "category": "technical",
            "urgency": "high",
            "summary": "Customer’s Wi-Fi drops about every 20 minutes since yesterday."
        })
    if "Draft a step-by-step troubleshooting plan" in last:
        return (
            "1) Power-cycle modem and router (unplug 30s, replug).\n"
            "2) Reseat coax/ethernet cables; check for damage.\n"
            "3) NimbusNet portal → Devices → Router → run line test.\n"
            "4) Temporarily disable QoS/traffic shaping to isolate config issues.\n"
            "5) If drops persist, schedule a technician to check signal levels."
        )
    if "Write a friendly response to the customer" in last:
        return (
            "Hi! Sorry about the drops—let’s stabilize this fast.\n\n"
            "Try these steps:\n"
            "1) Power-cycle your modem and router (unplug 30s, then replug).\n"
            "2) Reseat all cables (coax/ethernet) and ensure they click in.\n"
            "3) In the NimbusNet portal → Devices → Router → run a line test.\n"
            "4) If you use QoS, toggle it off briefly to test behavior.\n"
            "5) If it still drops, I can book a tech visit right away.\n\n"
            "Reply here with results and I’ll take the next step."
        )
    if "short closing line" in last or "closing line" in last:
        return "Glad we could help—wishing you smooth streaming! 🙌"
    return "OK."

# ===================== CHAIN STEPS =====================
def detect_intent(user_issue: str) -> dict:
    """
    Step 1: Identify intent and summarize.
    Returns dict: {category, urgency, summary}
    """
    msg = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"""
Classify the user's problem and extract key fields.

User issue: \"\"\"{user_issue}\"\"\"

Return JSON with:
- "category": one of ["billing","technical","account","shipping","other"]
- "urgency": one of ["low","medium","high"]
- "summary": 1-sentence summary
Only return JSON.
"""}]
    out = call_llm(msg)
    # Strip possible code fences and parse JSON robustly
    out = re.sub(r"```(?:json)?|```", "", out).strip()
    try:
        data = json.loads(out)
    except json.JSONDecodeError:
        data = {"category": "other", "urgency": "medium", "summary": user_issue[:120]}
    return data

def troubleshoot(state: dict) -> str:
    """
    Step 2: Generate a numbered troubleshooting plan based on detected state.
    """
    msg = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"""
Given this classified issue:
{json.dumps(state, ensure_ascii=False)}

Draft a step-by-step troubleshooting plan (3–6 bullets) tailored to the issue and clearly numbered.
Avoid generic advice; reference NimbusNet specifics (router reset, plan limits, portal steps).
"""}]
    return call_llm(msg)

def craft_response(user_issue: str, plan: str, max_words: int = 140) -> str:
    """
    Step 3: Friendly customer response using the plan.
    """
    msg = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"""
Write a friendly response to the customer:
- Start with a 1-sentence acknowledgement referencing their issue: \"{user_issue}\"
- Include the numbered troubleshooting plan below under "Try these steps:"
- Offer a fallback/next-step if it fails (e.g., escalate ticket, schedule tech visit)
- Keep under {max_words} words.
Troubleshooting plan:
{plan}
"""}]
    return call_llm(msg, temperature=0.4)

def closing_message() -> str:
    """
    Step 4: Short, warm closing line.
    """
    msg = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": "Write a short closing line (<=20 words) that sounds warm and professional."}
    ]
    return call_llm(msg, temperature=0.7)

# ===================== ORCHESTRATOR =====================
@dataclass
class ChainResult:
    intent: dict
    plan: str
    reply: str
    closing: str

def run_chain(user_issue: str) -> ChainResult:
    intent = detect_intent(user_issue)
    plan = troubleshoot(intent)
    reply = craft_response(user_issue, plan)
    closing = closing_message()
    return ChainResult(intent=intent, plan=plan, reply=reply, closing=closing)

# ===================== DEMO =====================
# You can change this input and re-run the cell in Colab.
example_issue = "My internet drops every 20 minutes since yesterday, and I’m on a deadline."
result = run_chain(example_issue)

print("=== INTENT ===")
print(json.dumps(result.intent, indent=2, ensure_ascii=False))
print("\n=== PLAN ===")
print(result.plan)
print("\n=== REPLY ===")
print(result.reply)
print("\n=== CLOSING ===")
print(result.closing)

# ===================== HOW TO USE REAL MODEL =====================
# 1) Set USE_MOCK = False near the top
# 2) In Colab, run:
#    %pip install openai
#    import os; os.environ["OPENAI_API_KEY"] = "sk-..."
# 3) Re-run this cell.


=== INTENT ===
{
  "category": "technical",
  "urgency": "high",
  "summary": "Customer’s Wi-Fi drops about every 20 minutes since yesterday."
}

=== PLAN ===
1) Power-cycle modem and router (unplug 30s, replug).
2) Reseat coax/ethernet cables; check for damage.
3) NimbusNet portal → Devices → Router → run line test.
4) Temporarily disable QoS/traffic shaping to isolate config issues.
5) If drops persist, schedule a technician to check signal levels.

=== REPLY ===
Hi! Sorry about the drops—let’s stabilize this fast.

Try these steps:
1) Power-cycle your modem and router (unplug 30s, then replug).
2) Reseat all cables (coax/ethernet) and ensure they click in.
3) In the NimbusNet portal → Devices → Router → run a line test.
4) If you use QoS, toggle it off briefly to test behavior.
5) If it still drops, I can book a tech visit right away.

Reply here with results and I’ll take the next step.

=== CLOSING ===
Glad we could help—wishing you smooth streaming! 🙌


In [ ]:
# @title ReACT Code Generation Demo (Colab-ready)
# @markdown - Works in **mock mode** by default (no API key needed).
# @markdown - To use a real model: set `USE_MOCK = False`, then `pip install openai` and set `OPENAI_API_KEY`.

from __future__ import annotations
import os, re, sys, ast
from dataclasses import dataclass

# ======================= CONFIG =======================
USE_MOCK = True          # <- set to False to call OpenAI
OPENAI_MODEL = "gpt-4o-mini"
MAX_TOOL_LOOPS = 4

SYSTEM_PROMPT = """You are a careful assistant that uses a strict ReACT workflow.

Rules:
- Think step-by-step in lines that start with 'Thought:' (plain text).
- When you need to run Python, write exactly:
  Action: python
  <<<CODE>>>
  # Python code with no imports
  # Use only builtins like range, len, sum, min, max, print
  # If you produce a value, store it in a variable named 'result'
  <<<ENDCODE>>>
- Wait for an Observation from the tool, then continue with another Thought.
- Finish with a single line starting with 'Final:' that states your concise final answer.

Constraints:
- Do NOT import modules (no 'import', 'from', etc.).
- Keep code minimal and deterministic.
"""

USER_TEMPLATE = """You can use one tool:
- python: executes the code you provide between <<<CODE>>> and <<<ENDCODE>>> and returns stdout, and the value of 'result' if present.

Task:
{task}

Follow the exact format:

Thought: <your reasoning>
Action: python
<<<CODE>>>
# your python code here (no imports)
<<<ENDCODE>>>
Observation: <tool output>
Thought: <what it means / next step>
... (repeat if needed)
Final: <concise final answer>"""

# ======================= LLM HELPER =======================
def call_llm(messages, model=OPENAI_MODEL, temperature=0.2) -> str:
    """Return assistant text from either a real provider or a mock."""
    if USE_MOCK:
        return _mock_llm(messages)

    # ---- Real OpenAI call (optional) ----
    try:
        from openai import OpenAI
    except Exception as e:
        raise RuntimeError("Install `openai` first: %pip install openai") from e

    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("Please set OPENAI_API_KEY in the environment.")
    client = OpenAI(api_key=api_key)
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return resp.choices[0].message.content

def _mock_llm(messages) -> str:
    """Simple deterministic mock that follows the ReACT format for demo purposes."""
    # We will parse whether an Observation has been provided; if so, produce Final.
    transcript = "\n".join(m["content"] for m in messages if m["role"] in ("user","assistant"))
    if "Observation:" not in transcript:
        # First model turn: reason + produce code
        return (
            "Thought: I will compute the mean and population standard deviation step-by-step.\n"
            "Action: python\n"
            "<<<CODE>>>\n"
            "data = [12, 15, 14, 10, 19]\n"
            "n = len(data)\n"
            "mean = sum(data)/n\n"
            "var = sum((x-mean)**2 for x in data)/n  # population variance\n"
            "std = var ** 0.5\n"
            "print(f\"mean={mean}, std={std}\")\n"
            "result = (mean, std)\n"
            "<<<ENDCODE>>>\n"
        )
    else:
        # Second model turn: read the observation and finalize
        m = re.search(r"Observation:\s*(.*)", transcript, re.S)
        obs = m.group(1).strip() if m else ""
        return (
            f"Thought: The observation already contains computed statistics ({obs}). "
            "No further code is needed.\n"
            "Final: Computed mean and population std from the list."
        )

# ======================= PYTHON TOOL =======================
def run_python_safely(code: str) -> str:
    """
    Execute untrusted Python with a tiny allowlist of builtins.
    - No imports allowed (we pre-scan AST to reject Import/ImportFrom).
    - Captures stdout; returns stdout + 'result=' value if present.
    """
    # Parse and reject imports or other disallowed nodes
    tree = ast.parse(code)
    for node in ast.walk(tree):
        if isinstance(node, (ast.Import, ast.ImportFrom)):
            raise RuntimeError("Imports are not allowed.")
        # Optional: forbid exec/eval/etc. (we already restrict builtins)
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            if node.func.id in {"__import__", "eval", "exec", "open", "compile"}:
                raise RuntimeError(f"Call to {node.func.id} is not allowed.")

    # Minimal safe builtins
    allowed_builtins = {
        "range": range, "len": len, "sum": sum, "min": min, "max": max, "print": print,
        "abs": abs, "round": round, "enumerate": enumerate
    }
    g = {"__builtins__": allowed_builtins}
    l = {}

    from io import StringIO
    buf = StringIO()
    old = sys.stdout
    try:
        sys.stdout = buf
        exec(code, g, l)
    finally:
        sys.stdout = old

    stdout = buf.getvalue().strip()
    if "result" in l:
        if stdout:
            return f"{stdout}\nresult={l['result']!r}"
        return f"result={l['result']!r}"
    return stdout or "<no output>"

# ======================= REACT LOOP =======================
@dataclass
class ReactResult:
    final: str
    transcript: str

def react(task: str, max_loops: int = MAX_TOOL_LOOPS) -> ReactResult:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(task=task)}
    ]
    transcript = []

    for _ in range(max_loops):
        resp = call_llm(messages)
        transcript.append(f"[ASSISTANT]\n{resp}")
        messages.append({"role": "assistant", "content": resp})

        # Parse for Action/Code/Final
        final_match = re.search(r"\bFinal:\s*(.*)", resp, re.S)
        if final_match:
            return ReactResult(final=final_match.group(1).strip(), transcript="\n".join(transcript))

        action_match = re.search(r"Action:\s*python", resp)
        code_match = re.search(r"<<<CODE>>>(.*?)<<<ENDCODE>>>", resp, re.S)

        if action_match and code_match:
            code = code_match.group(1).strip()
            try:
                observation = run_python_safely(code)
            except Exception as e:
                observation = f"[tool-error] {type(e).__name__}: {e}"
            obs_text = f"Observation: {observation}"
            transcript.append(f"[TOOL]\n{obs_text}")
            messages.append({"role": "user", "content": obs_text})
            continue

        # If neither code nor final, stop to avoid hanging
        break

    return ReactResult(final="(Stopped without Final.)", transcript="\n".join(transcript))

# ======================= DEMO =======================
# You can change TASK and re-run this cell.
TASK = "Generate Python to compute the mean and population standard deviation of [12, 15, 14, 10, 19] and return them."

result = react(TASK)

print("=== FINAL ===")
print(result.final)
print("\n=== TRACE ===")
print(result.transcript)

# ======================= HOW TO USE REAL MODEL =======================
# 1) Set USE_MOCK = False at the top of this cell.
# 2) In Colab, run:
#    %pip install openai
#    import os; os.environ['OPENAI_API_KEY'] = 'sk-...'
# 3) Re-run the cell.


=== FINAL ===
<concise final answer>). No further code is needed.
Final: Computed mean and population std from the list.

=== TRACE ===
[ASSISTANT]
Thought: The observation already contains computed statistics (<tool output>
Thought: <what it means / next step>
... (repeat if needed)
Final: <concise final answer>). No further code is needed.
Final: Computed mean and population std from the list.


In [ ]:
# 3_self_reflection.py
from llm_helper import call_llm

SYSTEM = "You are a precise editor. Be concise, specific, and constructive."

CRITIC_PROMPT = """You're evaluating the following summary for clarity, accuracy, completeness, and concision.
Return JSON with keys:
- strengths: 1-3 short bullets
- issues: 2-5 concrete issues (quote problematic text when helpful)
- fix_plan: 3-6 bullet plan of improvements (specific edits, not generic advice)
Summary:
\"\"\"{summary}\"\"\"
Only return JSON.
"""

REWRITE_PROMPT = """Improve the summary using the critique below.
Rules:
- Keep it 90–140 words.
- Fix inaccuracies, tighten wording, improve structure.
- Preserve all critical facts.
Critique:
{critique}
Original Summary:
\"\"\"{summary}\"\"\"
Return only the improved summary (no preface).
"""

def self_refine(summary: str) -> dict:
    # Step 1: Critique
    critique = call_llm(
        [{"role":"system","content":SYSTEM},
         {"role":"user","content":CRITIC_PROMPT.format(summary=summary)}],
        temperature=0.2
    )
    # Step 2: Improve
    improved = call_llm(
        [{"role":"system","content":SYSTEM},
         {"role":"user","content":REWRITE_PROMPT.format(critique=critique, summary=summary)}],
        temperature=0.3
    )
    return {"critique": critique, "improved": improved}

if __name__ == "__main__":
    draft = ("NimbusNet’s new plan expands speeds but some users see unstable Wi-Fi and billing confusion. "
             "Support should remind customers to reboot. The rollout started last month across regions.")
    out = self_refine(draft)
    print("=== CRITIQUE ===\n", out["critique"])
    print("\n=== IMPROVED SUMMARY ===\n", out["improved"])
